## Please input your directory for the top level folder
folder name : SUBMISSION MODEL

In [1]:
dir_ = 'C:/Users/jekim/OneDrive/바탕 화면/m5-forecasting-accuracy/' # input only here

#### setting other directory

In [2]:
raw_data_dir = dir_+'2. data/'
processed_data_dir = dir_+'2. data/processed/'
log_dir = dir_+'4. logs/'
model_dir = dir_+'5. models/'

In [3]:
####################################################################################
###################### 2-1. nonrecursive model by store ############################
####################################################################################

In [4]:
cvs = ['private']
STORES = ['CA_1', 'CA_2', 'CA_3', 'CA_4', 'TX_1', 'TX_2', 'TX_3', 'WI_1', 'WI_2', 'WI_3']
CLUSTERS = [0,1,2,3,4,5,6,7,8,9,10,11,12,13]

In [5]:
from  datetime import datetime, timedelta
import gc
import numpy as np, pandas as pd
import lightgbm as lgb

import os, sys, gc, time, warnings, pickle, psutil, random

warnings.filterwarnings('ignore')

In [6]:
def reduce_mem_usage(df, verbose=False):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                       df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [7]:
FIRST_DAY = 710
remove_feature = ['id',
                  'state_id',
                  'store_id',
                   'item_id',
                   'dept_id',
                   'cat_id',
                  'date','wm_yr_wk','d','sales']

cat_var = ['item_id', 'dept_id','store_id', 'cat_id', 'state_id'] + ["event_name_1", "event_name_2", "event_type_1", "event_type_2"]
cat_var = list(set(cat_var) - set(remove_feature))

In [8]:
grid2_colnm = ['sell_price', 'price_max', 'price_min', 'price_std',
               'price_mean', 'price_norm', 'price_nunique', 'item_nunique',
               'price_momentum', 'price_momentum_m', 'price_momentum_y']

grid3_colnm = ['event_name_1', 'event_type_1', 'event_name_2',
               'event_type_2', 'snap_CA', 'snap_TX', 'snap_WI', 'tm_d', 'tm_w', 'tm_m',
               'tm_y', 'tm_wm', 'tm_dw', 'tm_w_end']

lag_colnm = [ 'sales_lag_28', 'sales_lag_29', 'sales_lag_30',
             'sales_lag_31', 'sales_lag_32', 'sales_lag_33', 'sales_lag_34',
             'sales_lag_35', 'sales_lag_36', 'sales_lag_37', 'sales_lag_38',
             'sales_lag_39', 'sales_lag_40', 'sales_lag_41', 'sales_lag_42',
             
             'rolling_mean_7', 'rolling_std_7', 'rolling_mean_14', 'rolling_std_14',
             'rolling_mean_30', 'rolling_std_30', 'rolling_mean_60',
             'rolling_std_60', 'rolling_mean_180', 'rolling_std_180']

mean_enc_colnm = [
    
#    'enc_store_id_dept_id_mean', 'enc_store_id_dept_id_std', 
#    'enc_item_id_state_id_mean', 'enc_item_id_state_id_std',
    'enc_item_id_mean', 'enc_item_id_std'


]

In [9]:
########################### Make grid
#################################################################################
def prepare_data(cluster):
    
    grid_1 = pd.read_pickle(processed_data_dir+"grid_part_1.pkl")
    grid_2 = pd.read_pickle(processed_data_dir+"grid_part_2.pkl")[grid2_colnm]
    grid_3 = pd.read_pickle(processed_data_dir+"grid_part_3.pkl")[grid3_colnm]

    grid_df = pd.concat([grid_1, grid_2, grid_3], axis=1)
    del grid_1, grid_2, grid_3; gc.collect()
    
    grid_df = grid_df[grid_df['tskm_10'] == cluster]
    grid_df = grid_df[grid_df['d'] >= FIRST_DAY]
    
    lag = pd.read_pickle(processed_data_dir+"lags_df_28.pkl")[lag_colnm]
    
    lag = lag[lag.index.isin(grid_df.index)]
    
    grid_df = pd.concat([grid_df,
                     lag],
                    axis=1)
    
    del lag; gc.collect()
    

    mean_enc = pd.read_pickle(processed_data_dir+"mean_encoding_df.pkl")[mean_enc_colnm]
    mean_enc = mean_enc[mean_enc.index.isin(grid_df.index)]
    
    grid_df = pd.concat([grid_df,
                         mean_enc],
                        axis=1)    
    del mean_enc; gc.collect()
    
    grid_df = reduce_mem_usage(grid_df)
    
    
    
    return grid_df

In [10]:
validation = {
    'cv1' : [1551, 1610],
    'cv2' : [1829,1857],
    'cv3' : [1857, 1885],
    'cv4' : [1885,1913],
    'public' : [1913, 1941],
    'private' : [1941, 1969]
}

### cv1 : 2015-04-28 ~ 2015-06-26

### cv2 : 2016-02-01 ~ 2016-02-28

### cv3 : 2016-02-29 ~ 2016-03-27

### cv4 : 2016-03-28 ~ 2016-04-24

In [11]:
########################### Model params
#################################################################################
lgb_params = {
                    'boosting_type': 'gbdt',
                    'objective': 'tweedie',
                    'tweedie_variance_power': 1.1,
                    'metric': 'rmse',
                    'subsample': 0.5,
                    'subsample_freq': 1,
                    'learning_rate': 0.015,
                    'num_leaves': 2**8-1,
                    'min_data_in_leaf': 2**8-1,
                    'feature_fraction': 0.5,
                    'max_bin': 100,
                    'n_estimators': 3000,
                    'boost_from_average': False,
                    'verbose': -1,
                    'seed' : 1995
                } 

In [12]:
########################### Train Models
#################################################################################

rmsse_bycv = dict()

for cv in cvs:
    print('cv : day', validation[cv])

    pred_list = []
    for cluster in CLUSTERS:

        print('cluster', str(cluster), 'start')
        grid_df = prepare_data(cluster)

        model_var = grid_df.columns[~grid_df.columns.isin(remove_feature)]

        tr_mask = (grid_df['d'] <= validation[cv][0]) & (grid_df['d'] >= FIRST_DAY)
        vl_mask = (grid_df['d'] > validation[cv][0]) & (grid_df['d'] <= validation[cv][1])

        train_data = lgb.Dataset(grid_df[tr_mask][model_var], 
                       label=grid_df[tr_mask]['sales'])

        valid_data = lgb.Dataset(grid_df[vl_mask][model_var], 
                           label=grid_df[vl_mask]['sales'])

        m_lgb = lgb.train(lgb_params, train_data, valid_sets = [valid_data, train_data], verbose_eval=100) 
        display(pd.DataFrame({'name':m_lgb.feature_name(),
                              'imp':m_lgb.feature_importance()}).sort_values('imp',ascending=False).head(25))
        
        model_name = model_dir+'non_recur_model_item_cluster14_'+str(cluster)+'.bin'
        pickle.dump(m_lgb, open(model_name, 'wb'))
        
        del grid_df, train_data, valid_data, m_lgb, tr_mask, vl_mask; gc.collect

cv : day [1941, 1969]
cluster 0 start
[100]	training's rmse: 1.09115	valid_0's rmse: 0.940002
[200]	training's rmse: 1.07645	valid_0's rmse: 0.972753
[300]	training's rmse: 1.07011	valid_0's rmse: 0.991725
[400]	training's rmse: 1.064	valid_0's rmse: 1.00117
[500]	training's rmse: 1.0586	valid_0's rmse: 1.007
[600]	training's rmse: 1.05411	valid_0's rmse: 1.0112
[700]	training's rmse: 1.04995	valid_0's rmse: 1.01346
[800]	training's rmse: 1.04602	valid_0's rmse: 1.01352
[900]	training's rmse: 1.04268	valid_0's rmse: 1.01469
[1000]	training's rmse: 1.03942	valid_0's rmse: 1.01435
[1100]	training's rmse: 1.03644	valid_0's rmse: 1.0147
[1200]	training's rmse: 1.03357	valid_0's rmse: 1.01495
[1300]	training's rmse: 1.03081	valid_0's rmse: 1.01483
[1400]	training's rmse: 1.02824	valid_0's rmse: 1.01503
[1500]	training's rmse: 1.02579	valid_0's rmse: 1.01525
[1600]	training's rmse: 1.02342	valid_0's rmse: 1.01472
[1700]	training's rmse: 1.02104	valid_0's rmse: 1.01407
[1800]	training's rmse:

,name,imp
21,tm_w,36474
9,item_nunique,35672
50,rolling_mean_180,35236
1,release,34389
51,rolling_std_180,33302
48,rolling_mean_60,32067
52,enc_item_id_mean,31480
53,enc_item_id_std,31126
49,rolling_std_60,31084
47,rolling_std_30,27213


cluster 1 start
[100]	training's rmse: 10.0074	valid_0's rmse: 9.89208
[200]	training's rmse: 8.34335	valid_0's rmse: 12.8184
[300]	training's rmse: 7.8304	valid_0's rmse: 13.5698
[400]	training's rmse: 7.52932	valid_0's rmse: 13.7698
[500]	training's rmse: 7.30328	valid_0's rmse: 13.8943
[600]	training's rmse: 7.11058	valid_0's rmse: 13.9687
[700]	training's rmse: 6.9477	valid_0's rmse: 13.9798
[800]	training's rmse: 6.79564	valid_0's rmse: 13.993
[900]	training's rmse: 6.66366	valid_0's rmse: 13.9841
[1000]	training's rmse: 6.53999	valid_0's rmse: 13.9787
[1100]	training's rmse: 6.42444	valid_0's rmse: 13.9425
[1200]	training's rmse: 6.3162	valid_0's rmse: 13.9376
[1300]	training's rmse: 6.21282	valid_0's rmse: 13.9991
[1400]	training's rmse: 6.11854	valid_0's rmse: 14.0232
[1500]	training's rmse: 6.02674	valid_0's rmse: 14.0266
[1600]	training's rmse: 5.93953	valid_0's rmse: 14.0245
[1700]	training's rmse: 5.85546	valid_0's rmse: 14.0049
[1800]	training's rmse: 5.77611	valid_0's rms

,name,imp
21,tm_w,22355
51,rolling_std_180,21697
50,rolling_mean_180,21529
49,rolling_std_60,19944
48,rolling_mean_60,17460
9,item_nunique,16899
47,rolling_std_30,16681
20,tm_d,14494
43,rolling_std_7,14196
45,rolling_std_14,13949


cluster 2 start
[100]	training's rmse: 12.2924	valid_0's rmse: 20.7579
[200]	training's rmse: 8.73868	valid_0's rmse: 27.9672
[300]	training's rmse: 8.24689	valid_0's rmse: 29.8491
[400]	training's rmse: 8.04664	valid_0's rmse: 30.3526
[500]	training's rmse: 7.88777	valid_0's rmse: 30.4788
[600]	training's rmse: 7.74783	valid_0's rmse: 30.663
[700]	training's rmse: 7.61718	valid_0's rmse: 30.7163
[800]	training's rmse: 7.51004	valid_0's rmse: 30.7756
[900]	training's rmse: 7.40751	valid_0's rmse: 30.7995
[1000]	training's rmse: 7.30721	valid_0's rmse: 30.8587
[1100]	training's rmse: 7.2175	valid_0's rmse: 30.9016
[1200]	training's rmse: 7.13201	valid_0's rmse: 30.9433
[1300]	training's rmse: 7.0488	valid_0's rmse: 30.9365
[1400]	training's rmse: 6.96952	valid_0's rmse: 30.9368
[1500]	training's rmse: 6.89322	valid_0's rmse: 30.9615
[1600]	training's rmse: 6.81999	valid_0's rmse: 30.9749
[1700]	training's rmse: 6.74725	valid_0's rmse: 30.983
[1800]	training's rmse: 6.67742	valid_0's rms

...

,name,imp
50,rolling_mean_180,39278
21,tm_w,37415
9,item_nunique,36435
52,enc_item_id_mean,35502
51,rolling_std_180,34395
1,release,34260
53,enc_item_id_std,34207
48,rolling_mean_60,29978
11,price_momentum_m,29708
49,rolling_std_60,29629


cluster 12 start
[100]	training's rmse: 1.5879	valid_0's rmse: 1.40054
[200]	training's rmse: 1.55669	valid_0's rmse: 1.5505
[300]	training's rmse: 1.54124	valid_0's rmse: 1.59508
[400]	training's rmse: 1.52578	valid_0's rmse: 1.61698
[500]	training's rmse: 1.51344	valid_0's rmse: 1.62501
[600]	training's rmse: 1.50203	valid_0's rmse: 1.63159
[700]	training's rmse: 1.49226	valid_0's rmse: 1.63533
[800]	training's rmse: 1.48338	valid_0's rmse: 1.63846
[900]	training's rmse: 1.47557	valid_0's rmse: 1.63944
[1000]	training's rmse: 1.46813	valid_0's rmse: 1.6397
[1100]	training's rmse: 1.46125	valid_0's rmse: 1.64148
[1200]	training's rmse: 1.45511	valid_0's rmse: 1.64015
[1300]	training's rmse: 1.44922	valid_0's rmse: 1.64113
[1400]	training's rmse: 1.44351	valid_0's rmse: 1.64081
[1500]	training's rmse: 1.43786	valid_0's rmse: 1.64165
[1600]	training's rmse: 1.43269	valid_0's rmse: 1.64167
[1700]	training's rmse: 1.42742	valid_0's rmse: 1.64233
[1800]	training's rmse: 1.42275	valid_0's r

,name,imp
9,item_nunique,36484
21,tm_w,35918
50,rolling_mean_180,35147
51,rolling_std_180,33575
1,release,32824
48,rolling_mean_60,30205
53,enc_item_id_std,30167
49,rolling_std_60,30155
52,enc_item_id_mean,28609
47,rolling_std_30,26086


cluster 13 start
[100]	training's rmse: 1.175	valid_0's rmse: 0.921825
[200]	training's rmse: 1.16086	valid_0's rmse: 0.948325
[300]	training's rmse: 1.15245	valid_0's rmse: 0.977269
[400]	training's rmse: 1.14148	valid_0's rmse: 0.998616
[500]	training's rmse: 1.1307	valid_0's rmse: 1.01235
[600]	training's rmse: 1.12143	valid_0's rmse: 1.02154
[700]	training's rmse: 1.11279	valid_0's rmse: 1.02594
[800]	training's rmse: 1.10771	valid_0's rmse: 1.03028
[900]	training's rmse: 1.10336	valid_0's rmse: 1.03213
[1000]	training's rmse: 1.09914	valid_0's rmse: 1.03184
[1100]	training's rmse: 1.095	valid_0's rmse: 1.03315
[1200]	training's rmse: 1.09162	valid_0's rmse: 1.03356
[1300]	training's rmse: 1.08904	valid_0's rmse: 1.03347
[1400]	training's rmse: 1.08611	valid_0's rmse: 1.03375
[1500]	training's rmse: 1.08346	valid_0's rmse: 1.0343
[1600]	training's rmse: 1.08107	valid_0's rmse: 1.03373
[1700]	training's rmse: 1.07872	valid_0's rmse: 1.03404
[1800]	training's rmse: 1.07627	valid_0's 

,name,imp
21,tm_w,38497
11,price_momentum_m,36426
50,rolling_mean_180,34342
9,item_nunique,33383
52,enc_item_id_mean,32621
53,enc_item_id_std,32206
51,rolling_std_180,31051
5,price_std,29272
48,rolling_mean_60,27648
49,rolling_std_60,26212
